In [1]:
from fastai import *
from fastai.text.all import *
from fastai.basics import *

In [2]:
import re
import pandas as pd

In [19]:
def get_wiki(path,lang):
    name = f'{lang}wiki'
    if (path/name).exists():
        print(f"{path/name} already exists; not downloading")
        #return

    xml_fn = f"{lang}wiki-latest-pages-articles.xml"
    zip_fn = f"{xml_fn}.bz2"

    if not (path/xml_fn).exists():
        print("downloading...")
        download_url(f'https://dumps.wikimedia.org/{name}/latest/{zip_fn}', path/zip_fn)
        print("unzipping...")
        bunzip(path/zip_fn)

    with working_directory(path):
        #if not (path/'wikiextractor').exists(): os.system('git clone https://github.com/attardi/wikiextractor.git')
        print("extracting...")
        os.system("python -m wikiextractor.WikiExtractor --processes 4 --no_templates " +
            f"--min_text_length 1800 --filter_disambig_pages --log_file log -b 100G -q {xml_fn}")
    shutil.move(str(path/'text/AA/wiki_00'), str(path/name))
    shutil.rmtree(path/'text')

In [20]:
def split_wiki(path,lang):
    dest = path/'docs'
    name = f'{lang}wiki'
    if dest.exists():
        print(f"{dest} already exists; not splitting")
        return dest

    dest.mkdir(exist_ok=True, parents=True)
    title_re = re.compile(rf'<doc id="\d+" url="https://{lang}.wikipedia.org/wiki\?curid=\d+" title="([^"]+)">')
    lines = (path/name).open()
    f=None

    for i,l in enumerate(lines):
        if i%100000 == 0: print(i)
        if l.startswith('<doc id="'):
            title = title_re.findall(l)[0].replace('/','_')
            if len(title)>150: continue
            if f: f.close()
            f = (dest/f'{title}.txt').open('w')
        else: f.write(l)
    f.close()
    return dest

In [13]:
config = Config()

In [14]:
data_path = config['data_path']

In [15]:
lang = 'cy'
name = f"{lang}wiki"
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

In [33]:
get_wiki(path, lang)

/Users/ifanjohnston/.fastai/data/cywiki/cywiki already exists; not downloading
downloading...
unzipping...
extracting...


In [16]:
path.ls()

(#7) [Path('/Users/ifanjohnston/.fastai/data/cywiki/cywiki-latest-pages-articles.xml.bz2'),Path('/Users/ifanjohnston/.fastai/data/cywiki/cywiki-latest-pages-articles.xml'),Path('/Users/ifanjohnston/.fastai/data/cywiki/cywiki'),Path('/Users/ifanjohnston/.fastai/data/cywiki/docs_tok'),Path('/Users/ifanjohnston/.fastai/data/cywiki/docs'),Path('/Users/ifanjohnston/.fastai/data/cywiki/log'),Path('/Users/ifanjohnston/.fastai/data/cywiki/wikiextractor')]

In [17]:
!head -n10 {path}/{name}

<doc id="3" url="https://cy.wikipedia.org/wiki?curid=3" title="Cymraeg">
Cymraeg

Aelod o'r gangen Frythonaidd o'r ieithoedd Celtaidd a siaredir yn frodorol yng Nghymru, gan Gymry a phobl eraill ar wasgar yn Lloegr, a chan gymuned fechan yn Y Wladfa, yr Ariannin yw'r Gymraeg (hefyd Cymraeg heb y fannod).

Yng Nghyfrifiad y DU (2011), darganfuwyd fod 19% (562,000) o breswylwyr Cymru (tair blwydd a throsodd) yn gallu siarad Cymraeg. O'r ffigwr hon, darganfuwyd fod 77% (431,000) yn gallu siarad, darllen, ac ysgrifennu'r iaith; dywedwyd 73% o breswylwyr Cymru (2.2 miliwn) fod dim sgiliau yn y Gymraeg ganddynt. Gellir cymharu hwn â chyfrifiad 2001, a ddarganfu fod 20.8% o'r boblogaeth yn gallu siarad Cymraeg, gyda 57% (315,000) o'r ffigwr hon yn dweud eu bod yn rhugl yn yr iaith.

Mae'r Gymraeg yn un o'r 55 o ieithoedd i gael eu cynnwys fel cyfarchiad ar Record Aur y Voyager er mwyn cynrychioli'r Ddaear yn rhaglen Voyager NASA a lansiwyd ym 1977. Mae pob cyfarchiad yn unigryw i'r iaith, a d

In [21]:
dest = split_wiki(path, lang)

/Users/ifanjohnston/.fastai/data/cywiki/docs already exists; not splitting


In [59]:
dest.ls()[:5]

(#5) [Path('/Users/ifanjohnston/.fastai/data/cywiki/docs/Heini Gruffudd.txt'),Path('/Users/ifanjohnston/.fastai/data/cywiki/docs/Mynyddfor.txt'),Path('/Users/ifanjohnston/.fastai/data/cywiki/docs/Mathrafal.txt'),Path('/Users/ifanjohnston/.fastai/data/cywiki/docs/Taxus baccata.txt'),Path('/Users/ifanjohnston/.fastai/data/cywiki/docs/New Jersey Transit.txt')]

In [3]:
os.getcwd()

'/Users/ifanjohnston/PycharmProjects/fastaiWelshLM/nbs'

In [10]:
learn.load('model/cy_wt.pth')

NameError: name 'learn' is not defined

In [68]:
TextDataLoaders??

In [24]:
text_data = TextDataLoaders.from_folder(dest, valid_pct = 0.2, is_lm=True)

In [58]:
text_data.show_batch()

,text,text_
0,"xxbos xxmaj grugiar goch \n\n xxmaj aderyn ac isrywogaeth o adar yw xxmaj grugiar goch ( sy'n enw benywaidd ; enw lluosog : grugieir cochion ) a adnabyddir hefyd gyda'i enw gwyddonol xxmaj lagopus lagopus scotica ; yr enw xxmaj saesneg arno yw "" red grouse "" . xxmaj mae'n perthyn i deulu'r xxmaj ffesantod ( lladin : "" phasianidae "" ) sydd yn urdd y "" galliformes "" . xxmaj ystyrir","xxmaj grugiar goch \n\n xxmaj aderyn ac isrywogaeth o adar yw xxmaj grugiar goch ( sy'n enw benywaidd ; enw lluosog : grugieir cochion ) a adnabyddir hefyd gyda'i enw gwyddonol xxmaj lagopus lagopus scotica ; yr enw xxmaj saesneg arno yw "" red grouse "" . xxmaj mae'n perthyn i deulu'r xxmaj ffesantod ( lladin : "" phasianidae "" ) sydd yn urdd y "" galliformes "" . xxmaj ystyrir y"
1,", xxmaj nannetta yn "" falstaff "" xxmaj giuseppe xxmaj verdi , xxmaj xxunk yn "" i xxunk xxunk "" gan xxmaj ermanno wolf - ferrari , a xxmaj musetta yn "" la bohème "" xxmaj giacomo xxmaj puccini . xxmaj fe bortreadodd hefyd rôl y teitl yn y première xxmaj eidalaidd o "" xxunk xxunk xxmaj xxunk "" gan xxmaj xxunk xxmaj xxunk yn xxmaj la xxmaj scala ym 1958 .","xxmaj nannetta yn "" falstaff "" xxmaj giuseppe xxmaj verdi , xxmaj xxunk yn "" i xxunk xxunk "" gan xxmaj ermanno wolf - ferrari , a xxmaj musetta yn "" la bohème "" xxmaj giacomo xxmaj puccini . xxmaj fe bortreadodd hefyd rôl y teitl yn y première xxmaj eidalaidd o "" xxunk xxunk xxmaj xxunk "" gan xxmaj xxunk xxmaj xxunk yn xxmaj la xxmaj scala ym 1958 . \n\n"
2,"yn gynt , fel bod y stêm yn xxunk wrth wthio'r piston . \n\n xxmaj os bydd lle i gredu fod dŵr yn y silindrau ( er enghraifft ar gychwyn pan fydd ager yn troi'n ddŵr wrth gysylltu â'r silindr oer , neu os daw dŵr drosodd gyda'r ager o'r bwyler ) , y mae'n rhaid agor y xxunk gollwng dŵr sydd yng ngwaelod y silindrau ( gweler uchod ) . \n\n\n","gynt , fel bod y stêm yn xxunk wrth wthio'r piston . \n\n xxmaj os bydd lle i gredu fod dŵr yn y silindrau ( er enghraifft ar gychwyn pan fydd ager yn troi'n ddŵr wrth gysylltu â'r silindr oer , neu os daw dŵr drosodd gyda'r ager o'r bwyler ) , y mae'n rhaid agor y xxunk gollwng dŵr sydd yng ngwaelod y silindrau ( gweler uchod ) . \n\n\n <"
3,", y bont wreiddiol xxmaj pont y xxmaj borth a godwyd gan xxmaj thomas xxmaj telford ym 1826 a'r un fwy , xxmaj pont xxmaj britannia sydd yn cysylltu'r xxup a55 â'r ynys ynghyd â xxmaj rheilffordd xxmaj arfordir xxmaj gogledd xxmaj cymru . xxmaj yng xxmaj nghyfrifiad 2011 roedd poblogaeth yr ynys yn xxunk . xxmaj mae'r canran o'r boblogaeth sy'n siarad rhywfaint o xxmaj gymraeg oddeutu 70 % , yn","y bont wreiddiol xxmaj pont y xxmaj borth a godwyd gan xxmaj thomas xxmaj telford ym 1826 a'r un fwy , xxmaj pont xxmaj britannia sydd yn cysylltu'r xxup a55 â'r ynys ynghyd â xxmaj rheilffordd xxmaj arfordir xxmaj gogledd xxmaj cymru . xxmaj yng xxmaj nghyfrifiad 2011 roedd poblogaeth yr ynys yn xxunk . xxmaj mae'r canran o'r boblogaeth sy'n siarad rhywfaint o xxmaj gymraeg oddeutu 70 % , yn ôl"
4,"ystod y 1970au , cydweithiodd hefyd ar chwe albwm o ganeuon gyda xxmaj pete xxmaj atkin : \n\n\n xxmaj roedd adfywiad o ddiddordeb yn y caneuon yn y 1990au hwyr , a sbardunwyd yn bennaf gan xxmaj steve xxmaj xxunk o restr bostio xxmaj rhyngrwyd "" midnight xxmaj voices "" yn 1997 . xxmaj arweiniodd hyn at cyhoeddi chwe albwm ar xxup cd rhwng 1997 a 2001 , yn ogystal â","y 1970au , cydweithiodd hefyd ar chwe albwm o ganeuon gyda xxmaj pete xxmaj atkin : \n\n\n xxmaj roedd adfywiad o ddiddordeb yn y caneuon yn y 1990au hwyr , a sbardunwyd yn bennaf gan xxmaj steve xxmaj xxunk o restr bostio xxmaj rhyngrwyd "" midnight xxmaj voices "" yn 1997 . xxmaj arweiniodd hyn at cyhoeddi chwe albwm ar xxup cd rhwng 1997 a 2001 , yn ogystal â pherfformiadau"
5,"peters yn ei lyfr "" xxunk on xxmaj chaos "" yn dweud fod yn rhaid i reolwyr fod yn barod i dderbyn newid parhaus yn y gwaith . xxmaj mae son am newid i wella pethau yn gallu ymddangos fel beirniadaeth o berfformiad ac ar

In [64]:
len(text_data.vocab), len(text_data.train_ds)

(58744, 5416)

In [22]:
language_model_learner??

In [32]:
learn = language_model_learner(text_data, AWD_LSTM, drop_mult=0.5,path='',pretrained_fnames='cy_wt')

FileNotFoundError: [Errno 2] No such file or directory: 'models/y.pkl'

In [69]:
bs = 64
lr = 1e-2
lr *= bs/48

In [6]:
path = Path()
(path/'model').ls()

(#4) [Path('model/welshlm.pkl'),Path('model/cy_wt.pth'),Path('model/cy_wt_vocab.pkl.pth'),Path('model/welshlm10epoch.pth')]

In [9]:
load_learner??

In [16]:
learn_inf = load_learner('model/export_p2.pkl')

ValueError: unsupported pickle protocol: 5

In [21]:
# learn_inf = load('model/welshlm10epoch.pth')